In [ ]:
from collections import namedtuple
from websocket import create_connection
from typing import List
from pyspark.sql.types import *
from pyspark.sql.window import Window
import datetime
import os
import json
import requests
import re
import pandas as pd
import pyspark.sql.functions as f
import time

In [ ]:
os.environ["BASE_URL"] = "https://valsys.iai.cppib.io/"
def authenticate(username, password):
    # make the request
    auth_url =  os.environ["BASE_URL"] + "users/login"
    headers = {
      'username': username,
      'password': password
    }

    # decode into an object and validate
    response = requests.request("GET", auth_url, headers=headers, data=None)
    auth_response = json.loads(response.text.encode('utf8'), object_hook=lambda d: namedtuple('X', d.keys())(*d.values()))
    if auth_response.status != "success":
        print("ERROR:", auth_response.message)#
        return
    
    # set access token as environment variable
    os.environ["TOKEN"] = auth_response.data.AccessToken

In [ ]:
class Fact(object):
  def __init__(self, uid: str, identifier: str, formula: str, period: float, value: float, fmt: str):
      self.uid = uid
      self.identifier = identifier
      self.formula = formula
      self.period = period
      self.value = value
      self.fmt = fmt

  @classmethod
  def from_json(cls, data):
      return cls(data["uid"],
                 data.get("identifier", ""),
                 data.get("formula", ""),
                 data.get("period", 0),
                 data.get("value", 0),
                 data.get("fmt", ""))


class LineItem(object):
    def __init__(self, uid: str, name: str, facts: [Fact], tags=None):
        self.uid = uid
        self.name = name
        self.facts = facts
        self.tags = tags

    def add_item(self, name, modelID, caseID):
        path = "https://valsys.iai.cppib.io/modeling/add-module"
        body = {
            "token": os.environ["TOKEN"],
            "caseID": caseID,
            "modelID": modelID,
            "name": name,
            "parentModuleID": self.uid,
        }
        params = {"Content-Type": "application/json", "Authorization": auth_token()}
        resp = requests.post(url=path, headers=params, data=json.dumps(body))
        if resp.status_code != 200:
            print(resp.json())
        child_modules = resp.json()["data"]["module"]["childModules"]
        for module in child_modules:
            if module["name"] == name:
                return Module.from_json(module)

    def apply_edits(self, modelID, caseID):
        path = "https://valsys.iai.cppib.io/modeling/edit-formula"
        body = {
            "token": os.environ["TOKEN"],
            "caseID": caseID,
            "modelID": modelID,
            "forecastIncrement": 1,
        }
        facts = []
        for cell in self.facts:
          facts.append({
            "uid": cell.uid,
            "formula": cell.formula,
            "period": cell.period,
            "identifier": cell.identifier
          })
    
        body["facts"] = facts
        params = {"Content-Type": "application/json", "Authorization": auth_token()}
        resp = requests.post(url=path, headers=params, data=json.dumps(body)) 
        if resp.status_code != 200:
            print(resp.json())
        print("Edit applied to:", self.name)
        
    def edit_format(self, modelID, caseID):
        path = "https://valsys.iai.cppib.io/modeling/edit-format"
        body = {
            "token": os.environ["TOKEN"],
            "caseID": caseID,
            "modelID": modelID,
            "forecastIncrement": 1,
        }
        facts = []
        for cell in self.facts:
          facts.append({
            "uid": cell.uid,
            "format": cell.fmt,
            "identifier": cell.identifier
          })
    
        body["facts"] = facts
        params = {"Content-Type": "application/json", "Authorization": auth_token()}
        resp = requests.post(url=path, headers=params, data=json.dumps(body)) 
        if resp.status_code != 200:
            print(resp.json())
        print("Format edit applied to:", self.name)
       
    @classmethod
    def from_json(cls, data):
        facts = []
        if data.get("facts"):
            facts = list(map(Fact.from_json, data["facts"]))
        return cls(data["uid"], data["name"], facts, data.get("tags", None))


class Module(object):
    def __init__(self, uid: str, name: str, module_start: int, line_items: List[LineItem], child_modules: List['Module']):
        self.uid = uid
        self.name = name
        self.module_start = module_start
        self.line_items = line_items
        self.child_modules = child_modules

    def find_module(self, name: str) -> 'Module':
        if self.child_modules is None:
            return None
        for child_module in self.child_modules:
            if child_module.name == name:
                return child_module
            target = child_module.find_module(name)
            if target is not None:
                return target
        return None

    def add_item(self, name, order, modelID, caseID):
        path = "https://valsys.iai.cppib.io/modeling/add-item"
        body = {
            "token": os.environ["TOKEN"],
            "caseID": caseID,
            "modelID": modelID,
            "name": name,
            "order": order,
            "moduleID": self.uid,
        }
        params = {"Content-Type": "application/json", "Authorization": auth_token()}
        resp = requests.post(url=path, headers=params, data=json.dumps(body))
        if resp.status_code != 200:
            print(resp.json())
        print("Added item:", name)
        module = Module.from_json(resp.json()["data"]["module"])
        for l in module.line_items:
            if l.name == name:
                return l

    def pull_items_from_tags(self, tags: [str]) -> [LineItem]:
        items = []
        if self.line_items is not None:
            for item in self.line_items:
                if item.tags is not None:
                    for tag in tags:
                        if tag in item.tags:
                            items.append(item)
        if self.child_modules is not None:
            for child_module in self.child_modules:
                tagged_items = child_module.pull_items_from_tags(tags)
                if tagged_items != None:
                    items += tagged_items
        for item in items:
            sorted_facts = sorted(item.facts, key=lambda f: f.period)
            item.facts = sorted_facts
        return items

    def pull_item_by_name(self, name: str) -> LineItem:
        items = []
        if self.line_items is not None:
            for item in self.line_items:
                if item.name == name:
                  return item  
        return None
      
    def add_child_module(self, name, modelID, caseID):
        path = "https://valsys.iai.cppib.io/modeling/add-module"
        body = {
            "token": os.environ["TOKEN"],
            "caseID": caseID,
            "modelID": modelID,
            "name": name,
#             "unlinked": True,
            "parentModuleID": self.uid,
        }
        params = {"Content-Type": "application/json", "Authorization": auth_token()}
        resp = requests.post(url=path, headers=params, data=json.dumps(body))
        if resp.status_code != 200:
            print(resp.json())
        child_modules = resp.json()["data"]["module"]["childModules"]
        print("Created New Module: {}".format(name))
        for module in child_modules:
            if module["name"] == name:
                return Module.from_json(module)

    @classmethod
    def from_json(cls, data):
        line_items = []
        if data.get("lineItems"):
            line_items = list(map(LineItem.from_json, data["lineItems"]))
        if data.get("childModules") is not None:
            child_modules = list(map(Module.from_json, data["childModules"]))
            return cls(data["uid"], data["name"], data["moduleStart"], line_items, child_modules)
        return cls(data["uid"], data["name"], data["moduleStart"], line_items, None)


class Case(object):
    def __init__(self, uid: str, start_period: float, case: str, modules: List[Module]):
        self.uid = uid
        self.case = case
        self.start_period = start_period
        self.modules = modules
        self.ticker = ""

    def set_ticker(self, ticker):
        self.ticker = ticker

    @classmethod
    def from_json(cls, data):
        modules = list(map(Module.from_json, data["modules"]))
        return cls(data["uid"], data["startPeriod"], data["case"], modules)

    def pull_module(self, name: str) -> Module:
        if self.modules is None:
            return None
        for module in self.modules:
            if module.name == name:
                return module
            target = module.find_module(name)
            if target is not None:
                return target
        return None

    def pull_items_from_tags(self, tags: [str]) -> [LineItem]:
        items = []
        for module in self.modules:
            target_items = module.pull_items_from_tags(tags)
            if target_items is not None:
                items += target_items
        return items


class Model(object):
    def __init__(self, uid: str, cases: List[Case]):
        self.uid = uid
        self.cases = cases

    @classmethod
    def from_json(cls, data):
        cases = list(map(Case.from_json, data["cases"]))
        return cls(data["uid"], cases)

    def pull_case(self, name: str) -> Case:
        for case in self.cases:
            if case.case == name:
                return case


In [ ]:
def pull_model_information(uid: str) -> str:
    """Pulls the first case uid in a model"""
    path = "https://valsys.iai.cppib.io/modeling/model-information"
    params = {"Content-Type": "application/json", "Authorization": auth_token(), "modelID": uid}
    resp = requests.get(url=path, headers=params)
    if resp.status_code != 200:
        print(resp.json())
    return resp.json()["data"]["model"]["cases"][0]["uid"]
  
def recalculate_model(uid: str):
    """Recalculates the model"""
    path = "https://valsys.iai.cppib.io/modeling/recalculate"
    params = {"Content-Type": "application/json", "Authorization": auth_token(), "uid": uid}
    resp = requests.get(url=path, headers=params)
    if resp.status_code != 200:
        print(resp.json())

def pull_case(uid: str) -> Case:
    path = "https://valsys.iai.cppib.io/modeling/case"
    params = {"Content-Type": "application/json", "Authorization": auth_token(), "caseID": uid}
    resp = requests.get(url=path, headers=params)
    if resp.status_code != 200:
        print(resp.json())
    case = Case.from_json(resp.json()["data"]["case"])
    return case
  
def auth_token():
    """Returns the authorization parameter for authentication"""
    return "Bearer " + os.environ["TOKEN"]
  
def remove_module(modelID, caseID, moduleID, parentModuleID, module_name):
  path = "https://valsys.iai.cppib.io/modeling/delete-module"
  body = {
      "token": os.environ["TOKEN"],
      "caseID": caseID,
      "modelID": modelID,
      "parentModuleID": parentModuleID,
      "uid": moduleID
  }
  params = {"Content-Type": "application/json", "Authorization": auth_token()}
#   print(body)
#   print(json.dumps(body))
  resp = requests.post(url=path, headers=params, data=json.dumps(body))
  if resp.status_code != 200:
    print(resp.json())
  else:
    print("removed module")

In [ ]:
# machine_model_df = spark.read.table("model_engine.valsys_machine_models_v2").selectExpr("keydriver_source as source", "model_id").distinct()
machine_model_df = spark.read.table("model_engine.valsys_machine_models_om").selectExpr("keydriver_source as source", "model_id").distinct()


final_om_df = spark.read.table("model_engine.operating_model_config")

w_func = Window.partitionBy("source").orderBy(f.col("last_updated").desc())
latest_timestamp_df = final_om_df.select("source", "last_updated").distinct()
latest_timestamp_df = latest_timestamp_df.withColumn("update_rank", f.rank().over(w_func)).filter("update_rank = 1").drop("update_rank")

# module_df = module_df.withColumn("model_id", f.when(f.col("ticker") == "ANGI US", f.lit("0x21fa898")).otherwise(f.col("model_id")))
final_om_df = final_om_df.join(latest_timestamp_df, ["source", "last_updated"])
module_df = final_om_df.join(machine_model_df, ["source"]).withColumn("period_year", f.col("period_year").cast(IntegerType())).filter("ticker = 'W US'")

# check for new source
# for tbl in list(spark.catalog.listTables("model_engine")):
#   if "valsys_modules" in tbl.name:
#     df2 = spark.read.table("model_engine.valsys_machine_models").select('ticker', 'keydriver_source', 'templateID').drop_duplicates()
#     df = df.join(df2, [df2.keydriver_source == df.source], how="leftanti")

In [ ]:
username = "overlord@cppib.com"
password = "DDIOverlord!23"

authenticate(username, password)

In [ ]:
parent_module_name = "Key Drivers (Input)"
module_name = "Operating Model"
model_info = [{"source": x.source, "model_id": x.model_id, "ticker": x.ticker} for x in module_df.select("source", "model_id", "ticker").distinct().collect()]

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2471821865463924> in <module> 
 1 parent_module_name = "Key Drivers (Input)" 
 2 module_name = "Operating Model" 
 ----> 3 model_info = [ { "source" : x . source , "model_id" : x . model_id , "ticker" : x . ticker } for x in module_df . select ( "source" , "model_id" , "ticker" ) . distinct ( ) . collect ( ) ] 

 NameError : name 'module_df' is not defined

In [ ]:
for mi in model_info:
  model_id = mi["model_id"]
#   model_id = "0x21fa898"
  print("processing model id: {}, ticker: {}".format(model_id, mi["ticker"]))
  case_id = pull_model_information(model_id)
  case = pull_case(case_id)
  
  is_module = case.pull_module(parent_module_name)
  om_module = case.pull_module(module_name)
  module_id = is_module.uid
  
  if om_module is not None:
    print("removing {} module".format(module_name))
    om_module_id = om_module.uid
    remove_module(model_id, case_id, om_module_id, module_id, module_name)
  
  time.sleep(1)

In [ ]:
module_pdf = module_df.toPandas().sort_values(by=["source", "item_order", "period_year"])
module_name = "Operating Model"
key_metrics = ["Revenue Growth, %", "Gross Margin, %", "SG&A / sales", "R&D / sales", "Capex / sales"]
key_metrics_format = {"fontWeight":"bold","fontStyle":"normal","textAlign":"right","textDecoration":"none","valFormat":"Percentage","unit":"Raw","decimalPlaces":1}
om_module_info = []

for m_info in model_info:
  #   model_id = "0x21fa898"
  model_id = m_info["model_id"]
  kd_source = m_info["source"]
  om_dict = dict()
  
  print("processing: {}".format(kd_source))
   
  model_pdf = module_pdf.loc[module_pdf["model_id"] == model_id]
  # Pull the first case uid
  case_id = pull_model_information(model_id)
  # Pull the case data, the case returned holds the model data packaged as a Case object
  case = pull_case(case_id)
  # Select the income statement module as we want to add a new module as a revenue driver
  root_module = case.pull_module(parent_module_name)
  
  #create OM Module
  om_module = root_module.add_child_module(module_name, model_id, case_id)
  
  item_pdf = model_pdf[["kpi_name", "item_order"]].drop_duplicates().sort_values(by=["item_order"])
  
  om_item_lst = []

  for i,v in item_pdf.iterrows():
    om_item_dict = dict()
    id_dict = dict()

    item_name = v["kpi_name"]
    item_order = v["item_order"]

    om_item_dict["kpi_name"] = item_name
    item_obj = om_module.add_item(item_name, item_order, model_id, case_id)

    if item_name in key_metrics:

      for idx, cell in enumerate(item_obj.facts):
        cell.fmt = json.dumps(key_metrics_format)
        item_obj.facts[idx] = cell

      print("Editing Format: {}".format(item_name))
      item_obj.edit_format(model_id, case_id)

    om_item_dict["item_obj"] = item_obj
    om_item_lst.append(om_item_dict)

  id_lst = []
  for omi in om_item_lst:
    kpi_name = omi["kpi_name"]
    item_obj = omi["item_obj"]

    tmp_pdf = model_pdf.loc[model_pdf["kpi_name"] == kpi_name].sort_values(by=["period_year"])

    if len(tmp_pdf.index) > 0:
      id_dict_lst = []
      for idx, cell in enumerate(item_obj.facts):

        for i,v in tmp_pdf.iterrows():
          if cell.period == int(v["period_year"]):
            cell.formula = v["formula"]
            print(cell.formula)

            if v["period_name"] != "reported" and kpi_name in key_metrics:
              id_info = dict()
              id_info["identifier"] = cell.identifier
              id_info["period_year"] = cell.period
              id_dict_lst.append(id_info)
        item_obj.facts[idx] = cell
      item_obj.apply_edits(model_id, case_id)

      if len(id_dict_lst) > 0:
        id_dict = dict()
        id_dict[kpi_name] = id_dict_lst
        id_lst.append(id_dict)

    if kpi_name in key_metrics:
      om_dict["source"] = kd_source
      om_dict["model_id"] = model_id
      om_dict["case_id"] = case_id
      om_dict["case"] = case
      om_dict["metric"] = kpi_name

      for i in id_lst:
        if kpi_name in i.keys():
          om_dict["id_info"] = i[kpi_name]

      om_module_info.append(om_dict)

  time.sleep(1)

In [ ]:
keydriver_pdf = (spark.read.table("model_engine.xl_config_raw_om")
                     .filter((f.col("item_description").isin(*key_metrics)) & (f.col("source_type") == 'Operating Model') & (f.col("forecast") == 1))
                     .select("source", "item_description", "hybrid_formula_clean", "case")
                     .orderBy("source", "item_description")
                     .distinct()
                     .toPandas())

kd_info = dict()
for idx, val in keydriver_pdf.iterrows():
  src = val["source"]
  metric = val["item_description"]
  case = val["case"]
  fmla = val["hybrid_formula_clean"]
  
  fmla_info = dict()
  fmla_info[case] = fmla
  
  if src not in kd_info.keys():
    kd_info[src] = dict()
  
  if src in kd_info.keys() and metric not in kd_info[src].keys():
    kd_info[src][metric] = dict()
  
  kd_info[src][metric].update(fmla_info)

In [ ]:
for omi in om_module_info:
  src = omi["source"]
  metric = omi["metric"]
  id_lst = omi["id_info"]
  case = omi["case"]
  keydriver_module = case.pull_module(parent_module_name)
  
  for li in ["Base", "Upside", "Downside"]:
    item_name = "{} ({})".format(metric, li)
    line_item = keydriver_module.pull_item_by_name(item_name)
    scenario = li.lower()
    if src in kd_info.keys() and metric in kd_info[src].keys():
      print("processing source: {}, metric: {}, case: {}".format(src, metric, scenario))

      fmla = kd_info[src][metric][scenario]
      for idx, cell in enumerate(line_item.facts):
        for il in id_lst:
          if cell.period == il["period_year"]:
            cell_fmla = il["identifier"]
            if fmla is not None and len(fmla) > 0:
              cell_fmla = "{} {}".format(il["identifier"], fmla)
            cell.formula = cell_fmla
            line_item.facts[idx] = cell
      line_item.apply_edits(omi["model_id"], omi["case_id"])
  recalculate_model(model_id)